In [ ]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
pd.options.mode.chained_assignment = None 
from datetime import datetime
import scipy.interpolate
import scipy.optimize as optimize
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

In [ ]:
df_full = pd.read_csv('/Users/runqizhao/Desktop/APM466/A1/bond_data_a1.csv')

In [ ]:
df_full['Maturity'] = pd.to_datetime(df_full['Maturity'])


selected_columns = ['ISIN', 'Issue Date', 'Maturity', 'coupon', 
                    '2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11', '2024-01-12', '2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19']
df = df_full[selected_columns]

for d in df:
    print(d)

time = ['2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11', '2024-01-12', '2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19']

In [ ]:
# generate data
df.dropna(inplace=True)

In [ ]:
data = []
data.append(df.iloc[:,0:5])
for i in range(5, 14):
    data.append(df.iloc[:, [0, 1, 2, 3, i]])

In [ ]:
for data_f in data:
    current_time = datetime.fromisoformat(data_f.columns.values[-1])
    time_to_maturity = []
    for date in data_f["Maturity"]:
        time_to_maturity.append((date - current_time).days)
    data_f["time to maturity"] = time_to_maturity

    accrued_interest = []
    for i, row in data_f.iterrows():
        coupon = float(row["coupon"].strip('%'))
        days = 182-row["time to maturity"] % 182
        accrued_interest.append((days * coupon) / 365)
    data_f["accrued interest"] = accrued_interest

    dirty_price = []
    for i, row in data_f.iterrows():
        dirty_price.append(row[-3] + row["accrued interest"])
    data_f["dirty price"] = dirty_price
    
    yield_list = []
    x_list = []
    for i, row in data_f.iterrows():
        time_to_maturity = row["time to maturity"]
        coupon_rate = float(row["coupon"].strip('%'))

        x_list.append(time_to_maturity / 365)

        days = []
        for n in range(0, int(time_to_maturity / 182) + 1):
            day = (time_to_maturity % 182) / 182 + n
            days.append(day)
        days = np.asarray(days)

        payment = np.asarray([coupon_rate / 2] * int(time_to_maturity / 182) + [coupon_rate / 2 + 100])

        def equation(y):
            return np.dot(payment, (1 + y / 2) ** (-days)) - row["dirty price"]

        yield_value = optimize.fsolve(equation, 0.05)[0]
        yield_list.append(yield_value)    
    data_f["yield"] = yield_list
    data_f["x"] = x_list

In [ ]:
# uninterpolated yield curve
labels = ['2024-01-08','2024-01-09','2024-01-10','2024-01-11','2024-01-12', '2024-01-15','2024-01-16','2024-01-17','2024-01-18','2024-01-19']
for i in range(len(data)):
    plt.plot(data[i]["x"], data[i]["yield"], label = labels[i])
    plt.legend(loc='best')
    plt.xlim(0,5)
    plt.ylim(0,0.05)
    plt.xlabel('Time to Maturity')
    plt.ylabel('Yield')
    plt.title('5years Yield Curve')

In [ ]:
def draw_interploation(input1, input2):
    year_interval = []
    for i in range(1, 11):
        year_interval.append(i * 0.5)
    
    output = np.poly1d(np.polyfit(input1, input2, 2))(year_interval)
    year_interval, output = np.asarray(year_interval), np.asarray(output)
    return year_interval, output

In [ ]:
def calculate_diff(y, t, npl, df, i, col):
    future_cash_flows = np.dot(t[:-1], np.exp(-(np.multiply(npl[0, :i], df["x"][:i]))))
    present_value = future_cash_flows + t[i] * np.exp(-y * col["x"])
    return present_value - col["dirty price"]

def draw_spot_rate(df):
    res = np.zeros([1,11])
    for i, col in df.iterrows():
        rate = float(col["coupon"].strip('%')) / 2
        if i == 0:
            dirty_price = col["dirty price"]
            res[0, i] = -np.log(dirty_price / (rate+100)) / col["x"]
        else:
            t = np.asarray([rate] * i + [rate + 100])
            diff = lambda y: calculate_diff(y, t, res, df, i, col)
            res[0, i] = optimize.fsolve(diff, .05)
    return res

for i in range(len(data)):
    plt.plot(data[i]["x"], draw_spot_rate(data[i]).squeeze(), label = labels[i])
    plt.legend(loc='best')
    plt.xlabel('Time to Maturity')
    plt.ylabel('Spot Rate')
    plt.xlim(0,5)
    plt.ylim(0,0.05)
    plt.title('Spot Rate Curve')

In [ ]:

def forward_rate(df):
    input1, input2 = df["x"], draw_spot_rate(df).squeeze()
    rate1, rate2 = draw_interploation(input1, input2)
    f = []
    for i in [3, 5, 7, 9]:
        fi = (rate2[i] * (i + 1) / 2 - rate2[1]) / ((i + 1) / 2 - 1)
        f.append(fi)
    return f


# forward curve
for i in range(len(data)):
    plt.plot(['1yr-1yr','1yr-2yr','1yr-3yr','1yr-4yr'], forward_rate(data[i]), label = labels[i])
    plt.legend(loc = 'best')
    plt.ylim(0,0.05)
    plt.xlabel('Time to Maturity')
    plt.ylabel('Forward Rate')
    plt.title('Forward Rate Curve')

In [ ]:
print(forward_rate(data[0]))

In [ ]:
def cov(df):
    l, value = np.zeros([5,9]), np.zeros([5,10])
    
    for i in range(len(df)):
        rate1, rate2 = draw_interploation(df[i]["x"], df[i]["yield"])
        for j, idx in enumerate(range(1, 10, 2)):
            value[j, i] = rate2[idx]
    
    for i in range(0, 9):
        for j in range(5):
            l[j, i] = np.log(value[j, i+1] / value[j, i])
    
    return np.cov(l), l

print('Daily Log-Returns of yield')
print(cov(data)[0])
e1, e2 = np.linalg.eig(cov(data)[0])
print("eigenvalue for Daily Log-Returns of yield:")
print(e1)
print("eigenvector for Daily Log-Returns of yield: ")
print(e2)

In [ ]:
def calcualte_matrix(df):
    output = np.zeros([4,10])
    for i in range(len(df)):
        output[:,i] = forward_rate(df[i])
    return output

print("Forward Rate's Covariance Matrix: ") 
print(np.cov(calcualte_matrix(data)))
u1, u2 = np.linalg.eig(np.cov(calcualte_matrix(data)))
print("Eigenvalue for Forward Rate's Covariance Matrix:")
print(u1)
print("Eigenvector for Forward Rate's Covariance Matrix")
print(u2)